# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [1]:
%pip install openai==1.2 tiktoken datasets session-info --quiet


Note: you may need to restart the kernel to use updated packages.


### Imports

In [6]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

In [7]:
session_info.show()

### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [8]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "7bxxa151080a4e00a99ec8c84e53cbxx",            #Replace it with your credentials
    "AZURE_OPENAI_ENDPOINT": "https://gptxx-mini-msft-xxxx.openai.azure.com/",  #Replace it with your credentials
    "AZURE_OPENAI_APIVERSION": "2024-02-15-preview", #Replace it with your credentials
    "CHATGPT_MODEL": "gpt-xx-mini-Biswa"             #Replace it with your credentials
}

In [9]:
# Write the configuration information into the config.json file
with open('config1.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


In [10]:
with open('config1.json', 'r') as az_creds:
    data = az_creds.read()

In [11]:
creds = json.loads(data)

In [12]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [13]:
chat_model_id = creds["CHATGPT_MODEL"]

In [14]:
print(chat_model_id)

gpt-4o-mini-Biswa


### Utilities

In [15]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-4o-mini")  #  gpt-3.5-turbo

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>
    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

**(A) Upload and read csv file (2 Marks)**

In [16]:
cs_reviews_df = pd.read_csv("courier-service_reviews.csv")
# Read CSV File Here

In [17]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [18]:
cs_reviews_df.sample(5)

,id,review,sentiment
114,115,My experience with ExpressWay Logistics was ma...,Negative
27,28,Planning a destination wedding is undoubtedly ...,Positive
33,34,I was impressed by the speed of delivery offer...,Negative
88,89,ExpressWay Logistics' comprehensive tracking s...,Positive
57,58,ExpressWay Logistics' delivery times are incon...,Negative


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

In [19]:
cs_reviews_df['sentiment'].value_counts()

sentiment
Positive    68
Negative    63
Name: count, dtype: int64

In [20]:
cs_reviews_df.shape

(131, 3)

**(C) Split the Dataset (2 Marks)**

In [21]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)



In [22]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [23]:
columns_to_select = ['review','sentiment']

In [25]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [95]:
gold_examples

'[{"review":"The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. The packaging seemed more than adequate to protect the contents during transit.","sentiment":"Positive"},{"review":"ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their fa

In [96]:
json.loads(gold_examples)[0]  #Json format

{'review': "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'Positive'}

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [28]:
user_message_template = """```{courier_service_review}```"""

**(A) Write Zero Shot System Message (3 Marks)**

In [97]:
zero_shot_system_message = """Classify the courier servcie reviews presented in input  and provide better services to customer by categorizing reviews into following  sentiments:
sentiment=['Positive' or 'Negative'].Courier servcie reviews will be delimited by triple backticks in the input.
Answer only 'Positive' or 'Negative'.Act as a Logistics and transpotation expert.Donot explain the answer.
"""
# Write zero shot system message here


**(B) Create Zero Shot Prompt (2 Marks)**

In [30]:
zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
# Create zero shot prompt to be input ready for completion function

In [98]:
num_tokens_from_messages(zero_shot_prompt)

83

**(C) Write Few Shot System Message (3 Marks)**

In [99]:
few_shot_system_message = """Classify the  courier servcie review presented in input into following  sentiments:
sentiment=['Positive' or 'Negative'].
Courier servcie review will be delimited by triple backticks in the input.
Answer only 'Positive' or 'Negative'.Nothing else.Donot explain the answer.
"""


Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [100]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

**(D) Create Examples For Few shot prompte (2 Marks)**

In [113]:
#gold_examples = create_examples(cs_gold_examples_df,5)

examples = create_examples(cs_examples_df,5)
# Create Examples

In [114]:
examples

'[{"review":"ExpressWay Logistics is unreliable and untrustworthy. They failed to deliver my parcel on my address and the customer support team was unapologetic and unwilling to assist me in resolving the issue.I did not receive my parcel and suffered a lot..","sentiment":"Negative"},{"review":"The customer support team at ExpressWay Logistics was unhelpful and unresponsive when I tried to inquire about the status of my parcel. I received no assistance or explanation for the delay in delivery.I will not recommend it to anyone.","sentiment":"Negative"},{"review":"The tracking system used by ExpressWay Logistics is outdated and unreliable. It often fails to provide accurate information about the status of my parcels, leaving me in the dark about their whereabouts.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' dedication to safety and security gives us peace of mind when shipping valuable or sensitive items. They have robust security measures in place to protect our shipments

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [115]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

**(E) Create Few Shot Prompt (2 Marks)**

In [116]:
few_shot_prompt = create_prompt(
    few_shot_system_message,
    examples,
    user_message_template
)
# Create Few shot prompt


In [117]:
num_tokens_from_messages(few_shot_prompt)

609

In [118]:
input_description = cs_examples_df.iloc[0:,1]

user_input = [
    {
        'role':'user',
        'content': user_message_template.format(courier_service_review = input_description)
    }
]

In [119]:
input_description.count()

104

In [120]:
response = client.chat.completions.create(
    model= chat_model_id ,
    messages=few_shot_prompt+user_input,
    temperature=0, # <- Note the low temperature
    max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
)
print(response.choices[0].message.content)

Negative


##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [121]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [],[]

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2# <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            #print(prediction) #uncomment to see LLM response or to debug
            
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input.strip())
           
        except Exception as e:
            continue
    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")
    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))
    return micro_f1_score


Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

**(A) Evaluate zero shot prompt (2 Marks)**

In [122]:
evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                            

0.9

**(B) Evaluate few shot prompt (2 Marks)**

In [123]:
evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                            

0.9

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [59]:
num_eval_runs = 5

In [60]:
zero_shot_performance = []
few_shot_performance = []

In [82]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    #zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|██        | 1/5 [00:21<01:24, 21.07s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 40%|████      | 2/5 [00:38<00:57, 19.22s/it]

...

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 80%|████████  | 4/5 [01:25<00:22, 22.19s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|██████████| 5/5 [01:50<00:00, 22.06s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [259]:
# Compute mean and standard deviation for zero-shot
zero_shot_mean = np.array(zero_shot_performance).mean()
zero_shot_std = np.array(zero_shot_performance).std()
print(f"Zero-Shot Mean score: {zero_shot_mean:.4f}, Standard Deviation: {zero_shot_std:.4f}")

Zero-Shot Mean score: 0.9048, Standard Deviation: 0.0000


In [260]:
# Compute mean and standard deviation for few-shot
few_shot_mean = np.array(few_shot_performance).mean()
few_shot_std = np.array(few_shot_performance).std()
print(f"Few-Shot Mean score: {few_shot_mean:.4f}, Standard Deviation: {few_shot_std:.4f}")

Few-Shot Mean score: 0.9048, Standard Deviation: 0.0000


In [261]:
len(cs_reviews_df)

# Count the number of positive and negative reviews
positive_reviews = len(cs_reviews_df[cs_reviews_df['sentiment'] == 'Positive'])
#print(positive_reviews)
negative_reviews = len(cs_reviews_df[cs_reviews_df['sentiment'] == 'Negative'])

# Calculate the percentage
positive_percentage = (positive_reviews /len(cs_reviews_df )) * 100
negative_percentage = (negative_reviews / len(cs_reviews_df)) * 100

# Print results
print(f"Positive reviews: {positive_percentage:.2f}%")
print(f"Negative reviews: {negative_percentage:.2f}%")

Positive reviews: 51.91%
Negative reviews: 48.09%


##**Step 5: Observation and Insights and Business perspective (3 Marks)**

( Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist ExpressWay Logistics in addressing the issues identified. )


 ############   OBSERVATIONS  ##############
 1. Post executing the code multiple times with different samples the dataset seems perfectly balanced .
 2. The positive review is 52% and negative review is 48%. 
3. The LLM model is able to capture the sentiment more accuractely as compared with manual labelling ,Accuracy of model predicted is  90.48% overall model accuracy.Example case from dataset where expected and actual ground truth difference observed  is below :
****************************************************************************************************
" The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit."
***************************************************************************************************
4. The Zero shot prompt and few shot with examples both gives same accuracy .So we can use either of them.
But time taken by Zero shot is less as compared with Few shot by 2secs. 
5. We can go ahead and use the model to deploy in production . Post this step, a dashboard can be prepared to show the most frequent labels we have found. This will reveal the most frequent problem encountered by courier service.
